# Necessary Libraries

In [1]:
import io
import random
import string # to process standard python strings
import warnings
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')

In [2]:
pip install nltk


  Obtaining dependency information for nltk from https://files.pythonhosted.org/packages/a6/0a/0d20d2c0f16be91b9fa32a77b76c60f9baf6eba419e5ef5deca17af9c582/nltk-3.8.1-py3-none-any.whl.metadata
  Obtaining dependency information for click from https://files.pythonhosted.org/packages/00/2e/d53fa4befbf2cfa713304affc7ca780ce4fc1fd8710527771b58311a3229/click-8.1.7-py3-none-any.whl.metadata
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
  Obtaining dependency information for regex>=2021.8.3 from https://files.pythonhosted.org/packages/7f/61/b60849cd13f26a25b7aa61a0118c86c76b933a032d38b4657f4baeaeab5b/regex-2024.7.24-cp39-cp39-win_amd64.whl.metadata
     ---------------------------------------- 0.0/41.5 kB ? eta -:--:--
     ---------------------------------------- 41.5/41.5 kB ? eta 0:00:00
  Obtaining dependency information for tqdm from https://files.pythonhosted.org/packages/18/eb/fdb7eb9e48b7b02554e1664afd3bd3f117f6b6d6c5881438a0b055554f9b/tqdm-4.66.4-py3-none-any.whl.met


[notice] A new release of pip is available: 23.2.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('popular', quiet=True) # for downloading packages
#nltk.download('punkt') # first-time use only
#nltk.download('wordnet') # first-time use only

True

# Reading in the corpus
For our example,we will be using the Wikipedia page for chatbots as our corpus. Copy the contents from the page and place it in a text file named ‘chatbot.txt’. However, you can use any corpus of your choice.

In [4]:
f=open('chatbot.txt','r',errors = 'ignore')
raw=f.read()
raw = raw.lower()# converts to lowercase

## Tokenization

In [5]:
sent_tokens = nltk.sent_tokenize(raw)# converts to list of sentences
word_tokens = nltk.word_tokenize(raw)# converts to list of words

## Pre-Processing

In [6]:
lemmer = nltk.stem.WordNetLemmatizer()
#WordNet is a semantically-oriented dictionary of English included in NLTK.
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

# Keyword Matching

In [7]:
GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up","hey",)
GREETING_RESPONSES = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me"]
def greeting(sentence):

    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

# Generating response

In [8]:
def response(user_response):
    robo_response=''
    sent_tokens.append(user_response)
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
    tfidf = TfidfVec.fit_transform(sent_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf==0):
        robo_response=robo_response+"I am sorry! I don't understand you"
        return robo_response
    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response

In [9]:
flag=True
print("ROBO: My name is Robo. I will answer your queries about Chatbots. If you want to exit, type Bye!")
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("ROBO: You are welcome..")
        else:
            if(greeting(user_response)!=None):
                print("ROBO: "+greeting(user_response))
            else:
                print("ROBO: ",end="")
                print(response(user_response))
                sent_tokens.remove(user_response)
    else:
        flag=False
        print("ROBO: Bye! take care..")

ROBO: My name is Robo. I will answer your queries about Chatbots. If you want to exit, type Bye!
ROBO: hi there
ROBO: I am sorry! I don't understand you
ROBO: I am sorry! I don't understand you
ROBO: the notoriety of turing's proposed test stimulated great interest in joseph weizenbaum's program eliza, published in 1966, which seemed to be able to fool users into believing that they were conversing with a real human.
ROBO: interface designers have come to appreciate that humans' readiness to interpret computer output as genuinely conversational—even when it is actually based on rather simple pattern-matching—can be exploited for useful purposes.
ROBO: design
the chatbot design is the process that defines the interaction between the user and the chatbot.the chatbot designer will define the chatbot personality, the questions that will be asked to the users, and the overall interaction.it can be viewed as a subset of the conversational design.
ROBO: background
in 1950, alan turing's famou

# Improving the Chatbot using transformers

In [10]:
%pip install transformers
%pip install torch
%pip install flask

  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/13/63/cccd0297770d7096c19c99d4c542f3068a30e73cdfd971a920bfa686cb3a/transformers-4.43.2-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/43.7 kB ? eta -:--:--
     ---------------------------- ----------- 30.7/43.7 kB ? eta -:--:--
     ----------------------------------- -- 41.0/43.7 kB 393.8 kB/s eta 0:00:01
     -------------------------------------- 43.7/43.7 kB 355.5 kB/s eta 0:00:00
  Obtaining dependency information for filelock from https://files.pythonhosted.org/packages/ae/f0/48285f0262fe47103a4a45972ed2f9b93e4c80b8fd609fa98da78b2a5706/filelock-3.15.4-py3-none-any.whl.metadata
  Obtaining dependency information for huggingface-hub<1.0,>=0.23.2 from https://files.pythonhosted.org/packages/93/14/6a82b1c2eab5a828f7d3d675811660eb68424e8b039191f418a94e8d9726/huggingface_hub-0.24.2-py3-none-any.whl.metadata
  Obtaining dependency information for pyyaml>=5.1 fro


[notice] A new release of pip is available: 23.2.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


  Obtaining dependency information for torch from https://files.pythonhosted.org/packages/9f/ef/13faff7ef5770cea29ef2c06e2b87d6f34697973aef5eea4234948b46c4d/torch-2.4.0-cp39-cp39-win_amd64.whl.metadata
  Obtaining dependency information for sympy from https://files.pythonhosted.org/packages/b2/fe/81695a1aa331a842b582453b605175f419fe8540355886031328089d840a/sympy-1.13.1-py3-none-any.whl.metadata
  Obtaining dependency information for jinja2 from https://files.pythonhosted.org/packages/31/80/3a54838c3fb461f6fec263ebf3a3a41771bd05190238de3486aae8540c36/jinja2-3.1.4-py3-none-any.whl.metadata
  Obtaining dependency information for mpmath<1.4,>=1.1.0 from https://files.pythonhosted.org/packages/43/e3/7d92a15f894aa0c9c4b49b8ee9ac9850d6e63b03c9c32c0367a13ae62209/mpmath-1.3.0-py3-none-any.whl.metadata
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ---------------------------------------- 0.0/198.0 MB ? eta -:--:--
   ---------------------------------------- 0.2/198.0 MB 3.7 M


[notice] A new release of pip is available: 23.2.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


  Obtaining dependency information for flask from https://files.pythonhosted.org/packages/61/80/ffe1da13ad9300f87c93af113edd0638c75138c42a0994becfacac078c06/flask-3.0.3-py3-none-any.whl.metadata
  Obtaining dependency information for itsdangerous>=2.1.2 from https://files.pythonhosted.org/packages/04/96/92447566d16df59b2a776c0fb82dbc4d9e07cd95062562af01e408583fc4/itsdangerous-2.2.0-py3-none-any.whl.metadata
  Obtaining dependency information for blinker>=1.6.2 from https://files.pythonhosted.org/packages/bb/2a/10164ed1f31196a2f7f3799368a821765c62851ead0e630ab52b8e14b4d0/blinker-1.8.2-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/101.7 kB ? eta -:--:--
   ------------------------------------ --- 92.2/101.7 kB 2.6 MB/s eta 0:00:01
   ---------------------------------------- 101.7/101.7 kB 2.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
%pip install pyngrok


  Obtaining dependency information for pyngrok from https://files.pythonhosted.org/packages/9e/09/8338c63fe2d92313b7819f38c235746689d88052b112d8717bf45dcb3570/pyngrok-7.2.0-py3-none-any.whl.metadata



[notice] A new release of pip is available: 23.2.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


# Code for Improved Response Generation

In [12]:
from transformers import pipeline

# Load a pre-trained model and tokenizer from Hugging Face
chatbot = pipeline("text2text-generation", model="microsoft/DialoGPT-medium")

def get_response(user_input):
    response = chatbot(user_input)
    return response.generated_responses[0]


The model 'GPT2LMHeadModel' is not supported for text2text-generation. Supported models are ['BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGeneration', 'MarianMTModel', 'MBartForConditionalGeneration', 'MT5ForConditionalGeneration', 'MvpForConditionalGeneration', 'NllbMoeForConditionalGeneration', 'PegasusForConditionalGeneration', 'PegasusXForConditionalGeneration', 'PLBartForConditionalGeneration', 'ProphetNetForConditionalGeneration', 'SeamlessM4TForTextToText', 'SeamlessM4Tv2ForTextToText', 'SwitchTransformersForConditionalGeneration', 'T5ForConditionalGeneration', 'UMT5ForConditionalGeneration', 'XLMProphetNetForConditionalGeneration'].


# Basic Context Management
 We can use a simple dictionary to manage context within a single conversation session.

In [13]:
context = {}

def manage_context(user_id, user_input):
    if user_id not in context:
        context[user_id] = []
    context[user_id].append(user_input)
    return context[user_id]


# Basic Web Interface using Flask
We'll create a simple Flask web application to interact with the chatbot.

In [16]:
from flask import Flask, request, jsonify, render_template_string
from transformers import BlenderbotSmallTokenizer, BlenderbotSmallForConditionalGeneration
from pyngrok import ngrok

app = Flask(__name__)

# Load a pre-trained BlenderbotSmall model and tokenizer from Hugging Face
tokenizer = BlenderbotSmallTokenizer.from_pretrained("facebook/blenderbot_small-90M")
model = BlenderbotSmallForConditionalGeneration.from_pretrained("facebook/blenderbot_small-90M")

context = {}

def manage_context(user_id, user_input):
    if user_id not in context:
        context[user_id] = []
    context[user_id].append(user_input)
    return context[user_id]

def get_response(user_id, user_input):
    user_context = manage_context(user_id, user_input)
    inputs = tokenizer(" ".join(user_context), return_tensors="pt")
    reply_ids = model.generate(**inputs)
    response = tokenizer.batch_decode(reply_ids, skip_special_tokens=True)[0]
    return response

@app.route("/")
def home():
    return render_template_string("""
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <title>Chatbot</title>
        <script src="https://code.jquery.com/jquery-3.6.0.min.js"></script>
    </head>
    <body>
        <h1>Hello World</h1>
        <div id="chat-box">
            <div id="chat-log"></div>
            <input type="text" id="user-input" placeholder="Type your message...">
            <button onclick="sendMessage()">Send</button>
        </div>

        <script>
            function sendMessage() {
                var user_id = Math.random().toString(36).substr(2, 9); // generate random user id
                var user_input = $("#user-input").val();
                $.post("/get", {msg: user_input, user_id: user_id}, function(data) {
                    $("#chat-log").append("<p>You: " + user_input + "</p>");
                    $("#chat-log").append("<p>Bot: " + data.response + "</p>");
                    $("#user-input").val("");
                });
            }
        </script>
    </body>
    </html>
    """)

@app.route("/get", methods=["POST"])
def get_bot_response():
    user_input = request.form["msg"]
    user_id = request.form["user_id"]
    response = get_response(user_id, user_input)
    return jsonify({"response": response})

if __name__ == "__main__":
    app.run(host='0.0.0.0', port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.29.109:5000
Press CTRL+C to quit


In [17]:
import time
# Import and configure ngrok
from pyngrok import ngrok

# Run the Flask app in the background
from threading import Thread

def run_app():
    app.run(host="0.0.0.0", port=5000)

thread = Thread(target=run_app)
thread.start()
!ngrok authtoken 2jjsfF9yK3J4iGA1dSyviGmEcrc_5SB4thdANHDjWwgqAW2hw

# Now try to connect again
public_url = ngrok.connect(addr='5000')
print(f"Public URL: {public_url}")

while True:
    time.sleep(10)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.29.109:5000
Press CTRL+C to quit


Authtoken saved to configuration file: C:\Users\surab\AppData\Local/ngrok/ngrok.yml
Public URL: NgrokTunnel: "https://f2d2-49-36-26-205.ngrok-free.app" -> "http://localhost:5000"


127.0.0.1 - - [25/Jul/2024 23:25:24] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Jul/2024 23:25:25] "GET /favicon.ico HTTP/1.1" 404 -


KeyboardInterrupt: 